In [1]:
import pyspark
import pyspark.sql.functions as f
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master("local") \
    .appName("spark-avro-json-sample") \
    .config('spark.hadoop.avro.mapred.ignore.inputs.without.extension', 'true') \
    .getOrCreate()

In [3]:
avro_df = spark.read.format("avro").load("/home/jovyan/examples/data-samples/product-data-capture/*/*/*/*/*/*")
json_df = avro_df.select(avro_df.Body.cast("string")).rdd.map(lambda x: x[0])

#define schema
from pyspark.sql.types import StructType, StructField, TimestampType, StringType
schema = StructType([StructField("shortSku", StringType(), True), StructField("userId", StringType(), True), \
                    StructField("receivedAt", TimestampType(), True)])
json_df = spark.read.json(json_df, schema = schema, timestampFormat="yyyy-MM-dd HH:mm:ss")

json_df.printSchema()
json_df.show()
type(json_df)

root
 |-- shortSku: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- receivedAt: timestamp (nullable = true)

+--------+-----------+--------------------+
|shortSku|     userId|          receivedAt|
+--------+-----------+--------------------+
| J132983|11193438098|2019-04-24 16:25:...|
| A132323|11193438098|2019-04-24 18:25:...|
| J132323|11193438098|2019-04-24 18:25:...|
|   K3827|    3938783|2019-04-24 16:35:...|
|   K3928|    3938783|2019-04-19 16:35:...|
|   K3827|    3938783|2019-04-24 16:35:...|
|  K43632|     111098|2019-04-24 16:35:...|
|   K3827|    3938783|2019-04-24 16:35:...|
|   Z3892|    3938783|2019-04-21 16:35:...|
|  q43876|     111098|2019-04-24 16:35:...|
|   K3827|    3938783|2019-04-24 16:35:...|
|   Z3892|    3938783|2019-04-21 16:35:...|
|   K3827|     111098|2019-04-24 16:35:...|
|  K43632|     111098|2019-04-24 16:35:...|
+--------+-----------+--------------------+



pyspark.sql.dataframe.DataFrame

In [15]:
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth
j = json_df
d = j.select(j.shortSku, dayofmonth(j.receivedAt).alias('day'))
d2 = d.groupby('shortSku').pivot('day').count().show()
#json_df = json_df.groupBy('shortSku').pivot(month(json_df.receivedAt)).count().show()

+--------+----+----+----+
|shortSku|  19|  21|  24|
+--------+----+----+----+
| A132323|null|null|   1|
|   K3928|   1|null|null|
|  q43876|null|null|   1|
| J132983|null|null|   1|
| J132323|null|null|   1|
|   Z3892|null|   2|null|
|  K43632|null|null|   2|
|   K3827|null|null|   5|
+--------+----+----+----+



AttributeError: 'NoneType' object has no attribute 'createOrReplaceTempView'

In [16]:
d.createOrReplaceTempView("tblProducts")

In [17]:
df2 = spark.sql("SELECT * from tblProducts")
df2.show()

+--------+---+
|shortSku|day|
+--------+---+
| J132983| 24|
| A132323| 24|
| J132323| 24|
|   K3827| 24|
|   K3928| 19|
|   K3827| 24|
|  K43632| 24|
|   K3827| 24|
|   Z3892| 21|
|  q43876| 24|
|   K3827| 24|
|   Z3892| 21|
|   K3827| 24|
|  K43632| 24|
+--------+---+



In [20]:
df2 = spark.sql("SELECT shortSku, count(shortSku) as cnt FROM tblProducts group by shortSku").show()

+--------+---+
|shortSku|cnt|
+--------+---+
| A132323|  1|
|   K3928|  1|
|  q43876|  1|
| J132983|  1|
| J132323|  1|
|   Z3892|  2|
|  K43632|  2|
|   K3827|  5|
+--------+---+



In [21]:
df2 = spark.sql("SELECT shortSku, count(*) as c from tblProducts group by shortSku having c > 1 ").show()

+--------+---+
|shortSku|  c|
+--------+---+
|   Z3892|  2|
|  K43632|  2|
|   K3827|  5|
+--------+---+



In [26]:
# Spark 2.0 is ANSI SQL:2003 compliant
spark.sql("SELECT shortSku, day, \
            CASE \
                WHEN day > 1 THEN 'more than one' \
                ELSE 'just one' \
            END AS countText from tblProducts group by shortSku, day").show()

+--------+---+-------------+
|shortSku|day|    countText|
+--------+---+-------------+
|   Z3892| 21|more than one|
|  q43876| 24|more than one|
| A132323| 24|more than one|
| J132323| 24|more than one|
| J132983| 24|more than one|
|   K3928| 19|more than one|
|  K43632| 24|more than one|
|   K3827| 24|more than one|
+--------+---+-------------+

